In [2]:
import copy, math, os, pickle, time, pandas as pd, numpy as np, scipy.stats as ss


In [2]:
DATA_FILEPATH     = 'data/mimic_iii/test_dump/all_hourly_data.pkl'
GAP_TIME          = 6  # In hours
WINDOW_SIZE       = 24 # In hours
SEED              = 1
ID_COLS           = ['subject_id', 'hadm_id', 'icustay_id']
GPU               = '1'

os.environ['CUDA_VISIBLE_DEVICES'] = GPU
np.random.seed(SEED)

In [3]:
%%time
all_data = pd.read_pickle(DATA_FILEPATH)
data_full_lvl2 = all_data['vitals_labs']
statics        = all_data['patients']

Wall time: 3.2 s


In [4]:
print(f'Level2 data size: {data_full_lvl2.shape}')
print(f'statics: {statics.shape}')

Level2 data size: (2094295, 18)
statics: (29245, 29)


In [59]:
la_df =impute_mortality(data_full_lvl2)

C:\Users\cssar\miniconda3\envs\mimic_data_extraction_3.6\lib\site-packages\pandas\core\indexing.py:543: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self.obj[item] = s
C:\Users\cssar\miniconda3\envs\mimic_data_extraction_3.6\lib\site-packages\pandas\core\frame.py:4025: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  return super(DataFrame, self).rename(**kwargs)


In [58]:
def impute_mortality(df):
        idx = pd.IndexSlice
        df = df.copy()
        if len(df.columns.names) > 2: df.columns = df.columns.droplevel(('label', 'LEVEL1', 'LEVEL2'))
        
        df_out = df.loc[:, idx[:, ['mean', 'count']]]

        df_out.loc[:,idx[:,'mean']] = df_out.loc[:,idx[:,'mean']].groupby(ID_COLS).fillna(0)
        df_out.loc[:, idx[:, 'count']] = (df.loc[:, idx[:, 'count']] > 0).astype(float)
        df_out.rename(columns={'count': 'mask'}, level='Aggregation Function', inplace=True)

        is_absent = (1 - df_out.loc[:, idx[:, 'mask']])
        hours_of_absence = is_absent.cumsum()
        time_since_measured = hours_of_absence - hours_of_absence[is_absent==0].fillna(method='ffill')
        time_since_measured.rename(columns={'mask': 'time_since_measured'}, level='Aggregation Function', inplace=True)

        df_out = pd.concat((df_out, time_since_measured), axis=1)
        df_out.loc[:, idx[:, 'time_since_measured']] = df_out.loc[:, idx[:, 'time_since_measured']].fillna(100)
        
        df_out.sort_index(axis=1, inplace=True)
        return df_out

In [68]:
masked_data = la_df.loc[:,pd.IndexSlice[:, 'mask']].to_numpy()
missing_data = np.where(masked_data == 0)
masked_data.shape

(2094295, 6)

In [103]:
x_t = np.array([[1,2,3,8,7,9], [1,2,3,4,511,6], [1,2,3,4,5,6] ])
ldf = np.copy(x_t)
mask = np.where(ldf >3)
print(mask)
print(ldf[mask])
d = [ldf[x -1,i] if x != 0  else ldf[x,i] for x,i in zip(mask[0], mask[1])]
ldf[mask] = d
ldf

(array([0, 0, 0, 1, 1, 1, 2, 2, 2], dtype=int64), array([3, 4, 5, 3, 4, 5, 3, 4, 5], dtype=int64))
[  8   7   9   4 511   6   4   5   6]


array([[  1,   2,   3,   8,   7,   9],
       [  1,   2,   3,   8,   7,   9],
       [  1,   2,   3,   4, 511,   6]])

In [94]:
ldf.shape

(2, 6)

In [45]:
data_full_lvl2.loc[3]

LEVEL2                      diastolic blood pressure                         \
Aggregation Function                           count        mean        std   
hadm_id icustay_id hours_in                                                   
145834  211552     0                             6.0   39.666667  21.435174   
                   1                             8.0   44.125000  12.005207   
                   2                             6.0   47.333333  10.538817   
                   3                             4.0   64.500000   6.757712   
                   4                             1.0   63.000000        NaN   
                   5                             1.0   63.000000        NaN   
                   6                             1.0   70.000000        NaN   
                   7                             1.0   74.000000        NaN   
                   8                             1.0   71.000000        NaN   
                   9                             1.0   69.000000        NaN   
                   10                            1.0   59.000000        NaN   
                   11                            2.0   61.500000   6.363961   
                   12                            1.0   55.000000        NaN   
                   13                            1.0   56.000000        NaN   
                   14                            0.0         NaN        NaN   
                   15                            3.0   60.333333   3.055050   
                   16                            2.0   65.000000   1.414214   
                   17                            1.0   55.000000        NaN   
                   18                            2.0   64.500000   2.121320   
                   19                            1.0   62.000000        NaN   
                   20                            2.0   59.000000   0.000000   
                   21                            2.0   55.500000   2.121320   
                   22                            2.0   54.500000   3.535534   
                   23                            1.0   55.000000        NaN   
                   24                            1.0   52.000000        NaN   
                   25                            1.0   53.000000        NaN   
                   26                            1.0   51.000000        NaN   
                   27                            1.0   52.000000        NaN   
                   28                            1.0   51.000000        NaN   
                   29                            1.0   53.000000        NaN   
...                                              ...         ...        ...   
                   116                           1.0   55.000000        NaN   
                   117                           1.0   55.000000        NaN   
                   118                           4.0   65.750000   2.986079   
                   119                           1.0   57.000000        NaN   
                   120                           1.0   54.000000        NaN   
                   121                           1.0   47.000000        NaN   
                   122                           1.0   48.000000        NaN   
                   123                           1.0   39.000000        NaN   
                   124                           1.0   65.000000        NaN   
                   125                           1.0   65.000000        NaN   
                   126                           1.0   62.000000        NaN   
                   127                           1.0   56.000000        NaN   
                   128                           1.0   47.000000        NaN   
                   129                           1.0   51.000000        NaN   
                   130                           1.0   55.000000        NaN   
                   131                           1.0   55.000000        NaN   
                   132                           1

In [12]:
processed_data = np.load('../data/mimic_iii/preprocessed/mortality_and_los/decay_data_20926.npz')

In [17]:
# x_y_statics_xmean_mask_delta_lastob
x = processed_data['x']
y= processed_data['y']
statics = processed_data['statics']
x_mean = processed_data['xmean']
delta = processed_data['delta']
mask = processed_data['mask']
last_ob = processed_data['lastob']

In [19]:
print(f'x: {x.shape} y: {y.shape}, static:{statics.shape}, x_mean: {x_mean.shape}, delta:{delta.shape}, mask:{mask.shape}, last_ob: {last_ob.shape}')


x: (502224, 104) y: (20926, 1), static:(20926, 16), x_mean: (20926, 1, 104), delta:(502224, 104), mask:(502224, 104), last_ob: (502224, 104)
